# Chatbot 

Web-based interface of the chatbot interface: https://gonzalorecio.com/chatbot/robot.html

In [1]:
import requests
URL_INTERFACE = 'https://gonzalorecio.com/chatbot/robot.html'

In [2]:
class ChatbotFace:
    chatbot_mood_API   = 'https://chatbot-mood.herokuapp.com/mood'
    chatbot_status_API = 'https://chatbot-mood.herokuapp.com/internal_state'

    def change_mood(self, mood):
        req_obj = {'action': mood}
        x = requests.post(self.chatbot_mood_API, data = req_obj)
        # print(x)
        
    def change_status(self, status):
        req_obj = {'status': status}
        x = requests.post(self.chatbot_status_API, data = req_obj)
        
    def neutral(self):
        self.change_mood('neutral')
        
    def happy(self):
        self.change_mood('happy')
        
    def sad(self):
        self.change_mood('sad')
        
    def angry(self):
        self.change_mood('angry')
        
    def focused(self):
        self.change_mood('focused')
        
    def confused(self):
        self.change_mood('confused')
    
    def start_blinking(self):
        self.change_mood('start_blinking')
        
    def stop_blinking(self):
        self.change_mood('stopt_blinking')
        
    def status_listening(self):
        self.change_status('listening')
        
    def status_thinking(self):
        self.change_status('thinking')
    
    def status_recognizing(self):
        self.change_status('Recognizing')
    
    def status_custom(self, text):
        self.change_status(text)
    
    def status_none(self):
        self.change_status('')

In [3]:
chatbot = ChatbotFace()

In [4]:
chatbot.confused()

In [5]:
chatbot.happy()

In [6]:
chatbot.status_thinking()

In [7]:
chatbot.status_listening()

In [8]:
chatbot.status_custom('bon diaaaaa')

In [10]:
chatbot.status_none()

In [9]:
chatbot.neutral()

# Sentiment

In [13]:
from sentiments import sentiment_analysis as sent

Score: {'neg': 0.4, 'neu': 0.408, 'pos': 0.192, 'compound': -0.5499}
I am happy but today my mom died
Negative


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# Test
sent.sentiment_analysis("I've had a nice day today")

Score: {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}
I've had a nice day today
Positive


1

## Speech

In [11]:
from speech import google_speech as speech

In [13]:
# Test
speech.speech_to_audio(lang="en-US")

Listening...
Recognizing...
hola hola


'hola hola'

In [14]:
# Test
s = 'what are you saying?? :D'
speech.text_to_speech(s, speed=150, voice=1)

## Translation

In [12]:
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('สวัสดีจีน', lang_tgt='es')  
print(translate_text)
#output: Hello china

Hola china 


# Chatbot

In [20]:
import tensorflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google_trans_new import google_translator  

class Chatbot:
    def __init__(self, lang='en'):
        print('Loading model and tokenizers...')
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.face = ChatbotFace()
        
        self.reset_chatbot(lang=lang)
        print('Chatbot ready.')
    
    def reset_chatbot(self, lang='en'):
        self.chat_history_ids = self.tokenizer.encode(self.tokenizer.bos_token, return_tensors='pt')
        if lang=='es':
            self.voice = 0 # 0: spanish female, 1: english female, 2: english male
        else:
            self.voice = 1 # 0: spanish female, 1: english female, 2: english male
        self.translator = google_translator()  
        self.lang = lang
        
    def listen_and_get_question(self):
        self.face.status_listening()
        lang = 'en-US' if self.lang=='en' else 'es-ES'
        question = speech.speech_to_audio(chatbot_face=self.face)
        if self.lang=='es':
            question = self.translate(question, lang='en')
        self.face.status_none()
        return question
    
    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids, skip_special_tokens=True)
    
    def generate_answer(self, question):
        self.face.status_thinking()
        question_ids = self.tokenizer.encode(question + self.tokenizer.eos_token, return_tensors='pt')
        input_ids = torch.cat([self.chat_history_ids, question_ids], dim=-1)
        self.chat_history_ids = self.model.generate(input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)
        answer_ids = self.chat_history_ids[:, input_ids.shape[-1]:][0]
        text = self.decode(answer_ids)
        if self.lang == 'es':
            text = self.translate(text, lang='es')
        self.face.status_none()
        return text
    
    def get_sentiment_analysis(self, text):
        return True
        
    def speak(self, text):
        self.face.happy()
        speech.text_to_speech(text, speed=150, voice=self.voice)
        self.face.neutral()
        
    def translate(self, text, lang):
        translated_text = self.translator.translate(text, lang_tgt=lang)
        if type(translated_text) == list:
            return translated_text[0]
        return translated_text
    
    def no_understand(self):
        text = "Sorry, I couldn't understand you"
        if self.lang == 'es':
            text = self.translete(text, lang='es')
        self.face.status_custom(text)
        self.speak(text)
    
    def run_chat(self):
        self.reset_chatbot(lang=self.lang)
        self.face.neutral()
        question = ''
        while(question != 'goodbye'):
            question = self.listen_and_get_question()
            if question is None:
                print('Fallo')
                self.no_understand()
                continue
            print('Question:', question)
            answer = self.generate_answer(question)
            print('Answer:', answer)
            self.face.status_custom(answer)
            self.speak(answer)
            
#             self.reset_chatbot(lang=self.lang)
            print(self.chat_history_ids)
        

In [21]:
chatbot = Chatbot(lang='en')
# chatbot.run()

Loading model and tokenizers...
Chatbot ready.


In [21]:
chatbot.reset_chatbot(lang='en')

In [ ]:
chatbot.run_chat()

Listening...
Recognizing...
hello
Question: hello
Answer: Hello!
tensor([[50256, 31373, 50256, 15496,  5145, 50256]])
Listening...
Recognizing...
huawei
Question: huawei
Answer: Huawei
tensor([[50256, 31373, 50256, 15496,  5145, 50256, 13415, 38247, 50256, 38202,
         38247, 50256]])
Listening...
Recognizing...
how are you
Question: how are you
Answer: I'm good, how are you?
tensor([[50256, 31373, 50256, 15496,  5145, 50256, 13415, 38247, 50256, 38202,
         38247, 50256,  4919,   389,   345, 50256,    40,  1101,   922,   837,
           703,   389,   345,  5633, 50256]])
Listening...
Recognizing...
ampinchu what have you done today
Question: ampinchu what have you done today
Answer: I'm not sure, I'm not sure.
tensor([[50256, 31373, 50256, 15496,  5145, 50256, 13415, 38247, 50256, 38202,
         38247, 50256,  4919,   389,   345, 50256,    40,  1101,   922,   837,
           703,   389,   345,  5633, 50256,   696,  8589,    84,   644,   423,
           345,  1760,  1909, 50256